# Arbres k-dimensionnels

Ce notebook présente la recherche arborescente de plus proches voisins en dimension quelconque.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import load_digits, fetch_openml

## Données

### Disque

In [ ]:
def obtenir_disque(n_echantillons=100):
    '''Echantillons dans le carre unite; classe 1 si dans le disque unite.'''
    X = np.random.uniform(-1, 1, size=(n_echantillons, 2))
    y = (np.linalg.norm(X, axis=1) <= 1).astype(int)
    return X, y

In [ ]:
def montrer_disque(X, y):
    '''Montre les échantillons avec le disque.'''
    plt.figure(figsize=(4,4))
    plt.xlim(-1.1, 1.1)
    plt.ylim(-1.1, 1.1)
    x = np.linspace(-1, 1, 100)
    plt.plot(x, np.sqrt(1 - x**2), color='k')
    plt.plot(x, -np.sqrt(1 - x**2), color='k')
    plt.scatter(X[y==1, 0], X[y==1, 1])
    plt.scatter(X[y==0, 0], X[y==0, 1])
    plt.xticks([])
    plt.yticks([])

### Chiffres

In [ ]:
digits = load_digits()

In [ ]:
def obtenir_chiffres():
    '''Jeu de donnees Digits.'''    
    X = digits.data
    y = digits.target
    return X, y

In [ ]:
def montrer_chiffres(X, y, limit_max=10):
    '''Montre des chiffres.'''
    labels, nombres = np.unique(y, return_counts=True)
    nombre_max = min(np.max(nombres), limit_max)
    img = np.zeros((100, nombre_max*10))
    for i in range(10):
        index_label = np.where(y == i)[0][:limit_max]
        for j, echantillon in enumerate(index_label):
            img[i*10+1:i*10+9,j*10+1:j*10+9] = X[echantillon].reshape((8, 8))
    plt.imshow(img, cmap='binary')
    plt.xticks([])
    plt.yticks(5 + 10*np.arange(10), np.arange(10))

## Partage apprentissage / test

In [ ]:
def partager_apprentissage_test(X, y, ratio_test=0.2):
    '''Partage un jeu de données entre apprentissage et test. 
    La repartition entre classes est conservee.'''
    index = []
    for label in np.unique(y):
        index_label = np.where(y==label)[0]
        size = int(ratio_test * len(index_label))
        replace = size < len(index_label)
        index += list(np.random.choice(index_label, size=size, replace=replace))
    X_test = X[index]
    y_test = y[index]
    index_ = np.ones(len(y), dtype=bool)
    index_[index] = False
    X_app = X[index_]
    y_app = y[index_]
    return X_app, X_test, y_app, y_test

## Arbre k-dimensionnel

Recherche arborescente du plus proche voisin.

In [ ]:
class ArbreBoule:
    '''Arbre boule. Chaque noeud (sauf les feuilles) contient le centre et le rayon de la boule.'''
    def __init__(self, ancetre):
        self.ancetre = ancetre
        self.index = None
        self.centre = None
        self.rayon = None
        self.gauche = None
        self.droit = None

In [ ]:
def construire_arbre(X_app, profondeur=0, index=None, ancetre=None, taille_min=10):
    '''Construit un arbre de façon récursive.'''
    n_echantillons, n_dimensions = X_app.shape
    if profondeur == 0:
        index = np.arange(n_echantillons)
    arbre = ArbreBoule(ancetre)
    if len(index) > taille_min:
        # on divise la feuille
        axe = profondeur % n_dimensions
        median = len(index) // 2
        index_median = index[np.argpartition(X_app[index, axe], median)]
        centre = index_median[median]
        index_gauche = index_median[:median]
        index_droit = index_median[median + 1:]
        arbre.centre = centre
        arbre.rayon = np.max(np.linalg.norm(X_app[index] - X_app[centre], axis=1))
        arbre.gauche = construire_arbre(X_app, profondeur + 1, index_gauche, arbre, taille_min)
        arbre.droit = construire_arbre(X_app, profondeur + 1, index_droit, arbre, taille_min)
    else:
        arbre.index = index
    return arbre

In [ ]:
def rechercher_feuille(x_test, X_app, arbre, profondeur=0):
    '''Recherche récursive de la feuille où se trouve la cible (un seul échantillon).'''
    n_dimensions = len(x_test)
    if arbre.centre is not None:
        axe = profondeur % n_dimensions
        centre = arbre.centre
        if x_test[axe] <= X_app[centre, axe]:
            return rechercher_feuille(x_test, X_app, arbre.gauche, profondeur + 1)
        else:
            return rechercher_feuille(x_test, X_app, arbre.droit, profondeur + 1)
    else:
        return arbre

In [ ]:
def rechercher_plus_proche_voisin(x_test, X_app, arbre, distance=np.inf):
    '''Recherche récursive du plus proche voisin d'une cible (un seul échantillon).'''
    noeud = rechercher_feuille(x_test, X_app, arbre)
    index = noeud.index
    distances = np.linalg.norm(X_app[index] - x_test, axis=1)
    voisin = index[np.argmin(distances)]
    distance = np.minimum(np.linalg.norm(X_app[voisin] - x_test), distance)
    
    while noeud.ancetre is not None:  
        ancetre = noeud.ancetre
        # on teste le centre de la boule
        centre = ancetre.centre
        distance_centre = np.linalg.norm(X_app[centre] - x_test)
        if distance_centre < distance:
            voisin, distance = centre, distance_centre
        # on teste l'autre sous-arbre
        if noeud == ancetre.gauche:
            noeud = ancetre.droit
        else:
            noeud = ancetre.gauche 
        if noeud is not None:
            if not noeud.centre or np.linalg.norm(X_app[noeud.centre] - x_test) < distance + noeud.rayon:
                # on explore le sous-arbre
                noeud.ancetre = None
                voisin_, distance_ = rechercher_plus_proche_voisin(x_test, X_app, noeud, distance)
                if distance_ < distance:
                    voisin, distance = voisin_, distance_
                noeud.ancetre = ancetre
        # on remonte dans l'arbre
        noeud = ancetre
        
    return voisin, distance

In [ ]:
def rechercher_plus_proches_voisins(X_test, X_app, arbre):
    '''Recherche récursive des plus proches voisins. 
    Retourne un vecteur de taille n_test.'''
    voisins = []
    for x_test in X_test:
        voisin, _ = rechercher_plus_proche_voisin(x_test, X_app, arbre)
        voisins.append(voisin)       
    return voisins

In [ ]:
def classifier_plus_proches_voisins(X_test, X_app, y_app, arbre):
    '''Classifier par le plus proche voisin. 
    Retourne un vecteur de taille n_test.'''
    voisins = rechercher_plus_proches_voisins(X_test, X_app, arbre)    
    return y_app[voisins]

### Disque

In [ ]:
X, y = obtenir_disque(1000)

In [ ]:
X_app, X_test, y_app, y_test = partager_apprentissage_test(X, y)

In [ ]:
arbre = construire_arbre(X_app)

In [ ]:
feuille = rechercher_feuille([0, 0], X_app, arbre)

In [ ]:
montrer_disque(X_app[feuille.index], y_app[feuille.index])

In [ ]:
feuille = rechercher_feuille([1, 1], X_app, arbre)

In [ ]:
montrer_disque(X_app[feuille.index], y_app[feuille.index])

In [ ]:
y_pred = classifier_plus_proches_voisins(X_test, X_app, y_app, arbre)

In [ ]:
precision = np.sum(y_pred == y_test) / len(y_test)

In [ ]:
np.round(precision, 2)

### Chiffres

In [ ]:
X, y = obtenir_chiffres()

In [ ]:
X_app, X_test, y_app, y_test = partager_apprentissage_test(X, y)

In [ ]:
arbre = construire_arbre(X_app)

In [ ]:
echantillon = np.random.choice(len(X_test))
feuille = rechercher_feuille(X_test[echantillon], X_app, arbre)

In [ ]:
y_test[echantillon]

In [ ]:
montrer_chiffres(X_app[feuille.index], y_app[feuille.index])

In [ ]:
voisin, distance = rechercher_plus_proche_voisin(X_test[echantillon], X_app, arbre)

In [ ]:
y_app[voisin]

In [ ]:
y_pred = classifier_plus_proches_voisins(X_test, X_app, y_app, arbre)

In [ ]:
precision = np.sum(y_pred == y_test) / len(y_test)

In [ ]:
np.round(precision, 2)

## Quelques idées à explorer

* Étendre la recherche aux $k$ plus proches voisins.
* Donner la fraction des échantillons explorés pour une recherche.
* Coder un arbre avec exploration opportuniste des axes (et non cyclique comme dans le code actuel) ; par exemple, chaque coupe se fait sur l'axe de plus grande étendue (max - min des échantillons sur cet axe).